# SRA

In [3]:
import numpy as np
import math
import random

class SRA:
    def __init__(self, objf, lb, ub, dim, PopSize, iters, function_name="SRA"):
        self.objf = objf
        self.lb = lb
        self.ub = ub
        self.dim = dim
        self.PopSize = PopSize
        self.iters = iters
        self.function_name = function_name
        self.positions = np.zeros((PopSize, dim))
        self.best_position = np.zeros(dim)
        self.best_cost = float("inf")
        self.costs = np.full(PopSize, float("inf"))

    def initialize_positions(self):
        for i in range(self.dim):
            self.positions[:, i] = np.random.uniform(0, 1, self.PopSize) * (self.ub[i] - self.lb[i]) + self.lb[i]

    def evaluate_positions(self):
        for i in range(self.PopSize):
            self.positions[i, :] = np.clip(self.positions[i, :], self.lb, self.ub)
            cost = self.objf(self.positions[i, :])
            self.costs[i] = cost
            # Update the best solution found
            if cost < self.best_cost:
                self.best_cost = cost
                self.best_position = self.positions[i, :].copy()

    def update_positions(self, l):
        L = 0.5
        h = 6.625 * 10**-34
        b = 1 - (l ** (1.0 / 5)) / (self.iters ** (1.0 / 5))  # Decreasing factor

        # Sort and get indices based on cost
        SmellOrder = np.sort(self.costs)
        SmellIndex = np.argsort(self.costs)
        sorted_positions = self.positions[SmellIndex, :]

        # Update best and worst cost
        Best_X = sorted_positions[0, :]
        Best_Cost = SmellOrder[0]
        Worst_Cost = SmellOrder[-1]

        for i in range(self.PopSize):
            # Probability for Particle-Like vs. Wave-Like
            Seq = np.array(range(0, self.PopSize))
            R = self.PopSize - Seq
            p = (R / self.PopSize)**2
            h2 = p[i]

            vc = np.random.uniform(-b, b, self.dim)
            Z = self.Levy(self.dim)

            if random.random() < 0.03:
                Xnew = np.random.uniform(0, 1, self.dim) * (np.array(self.ub) - np.array(self.lb)) + np.array(self.lb)
            else:
                ids_except_current = [_ for _ in range(self.PopSize) if _ != i]
                id_1, id_2 = random.sample(ids_except_current, 2)

                # Particle-Like
                if abs(p[i]) >= 0.5:
                    if np.random.rand() < 0.5:
                        Xnew = Best_X + np.random.rand() * ((np.array(self.ub) - np.array(self.lb)) * np.random.rand() + np.array(self.lb))
                    else:
                        Xnew = self.positions[i, :] - Z + vc
                # Wave-Like
                else:
                    pos_1 = Best_X + np.random.rand() * vc * (h * (Best_X - Worst_Cost) + (h2 * (self.positions[id_1, :] - 2 * self.positions[i, :] + self.positions[id_2, :])))
                    pos_2 = self.positions[i, :] + np.random.rand() * vc * (h * (Best_X - Worst_Cost) + (h2 * (self.positions[id_1, :] + 2 * self.positions[i, :] + self.positions[id_2, :])))
                    Xnew = np.where(np.random.uniform(0, 1, self.dim) < p[i], pos_1, pos_2)

            # Apply bounds and evaluate new position
            Xnew = np.clip(Xnew, self.lb, self.ub)
            Xnew_cost = self.objf(Xnew)

            # Update the position if new position is better
            if self.costs[i] > Xnew_cost:
                self.costs[i] = Xnew_cost
                self.positions[i, :] = Xnew
                if Xnew_cost < self.best_cost:
                    self.best_position = Xnew
                    self.best_cost = Xnew_cost

    def Levy(self, dim):
        beta = 1.5
        sigma = (math.gamma(1 + beta) * math.sin(math.pi * beta / 2) / (math.gamma((1 + beta) / 2) * beta * 2 ** ((beta - 1) / 2))) ** (1 / beta)
        u = 0.01 * np.random.randn(dim) * sigma
        v = np.random.randn(dim)
        zz = np.power(np.abs(v), (1 / beta))
        step = np.divide(u, zz)
        return step

    def optimize(self):
        self.initialize_positions()
        self.evaluate_positions()
        convergence_curve = np.zeros(self.iters)
        print("SRA is Optimize    ")
        for l in range(self.iters):
            self.update_positions(l)
            convergence_curve[l] = self.best_cost

            # Optional: print progress
            # if l % 100 == 0:
            #     print(f"Iteration {l}, Best Cost: {self.best_cost}")

        return self.best_position, self.best_cost, convergence_curve


# Define the G10 Fitness Function with Constraints
def g10_fitness(x):
    """
    Calculate the fitness value and constraints of G10.
    
    Parameters:
        x (array): Solution vector x with 8 decision variables.
        
    Returns:
        f (float): Fitness value of the solution.
        g (list): Constraint violation values (inequality constraints).
    """
    # Objective function
    f = x[0] + x[1] + x[2]
    
    # Constraints
    g = [
        -1 + 0.0025 * (x[3] + x[5]),
        -1 + 0.0025 * (x[4] + x[6] - x[3]),
        -1 + 0.01 * (x[7] - x[4]),
        -x[0] * x[5] + 833.33252 * x[3] + 100 * x[0] - 83333.333,
        -x[1] * x[6] + 1250 * x[4] + x[1] * x[3] - 1250 * x[3],
        -x[2] * x[7] + 1250000 + x[2] * x[4] - 2500 * x[4]
    ]
    
    return f, g

# Objective Function with Penalty for Constraints
def evaluate(x):
    f, g = g10_fitness(x)
    penalty = 1e6 * sum(max(0, violation) for violation in g)  # Penalty for any constraint violation
    return f + penalty


# G10 Problem Parameters
dim = 8
lb = [100, 1000, 1000, 10, 10, 10, 10, 10]
ub = [10000, 10000, 10000, 1000, 1000, 1000, 1000, 1000]
PopSize = 50
iters = 2000
runs = 30

# Run SRA on G10 and count constraint violations
constraint_violations = 0

for run in range(runs):
    # Initialize and run GWO optimization
    sra = SRA(evaluate, lb, ub, dim, PopSize, iters, function_name="G10")
    best_position, best_fitness,_ = sra.optimize()
    print("best_position, best_fitness   ",  best_position, best_fitness)
    # Check constraint violations in the best solution found
    _, g = g10_fitness(best_position)
    
    # Count if any constraint is violated
    if any(violation > 0 for violation in g):
        constraint_violations += 1

print(f"Out of {runs} runs, constraints were violated {constraint_violations} times.")

SRA is Optimize    
best_position, best_fitness    [1568.41053668 1000.0009513  4886.49682729  228.76958729  304.64257317
  170.83366224  324.1179632   404.6272516 ] 7454.908315269178
SRA is Optimize    
best_position, best_fitness    [ 100.         3131.76526389 4676.46717839   95.14219285  312.96381877
  182.14861251  182.17324708  412.96170571] 7908.2324422849015
SRA is Optimize    
best_position, best_fitness    [ 100.         1000.0058203  6554.30908038  108.89946393  237.92395218
  177.78722657  270.21909994  337.92069251] 7654.314900675101
SRA is Optimize    
best_position, best_fitness    [ 100.00000326 1000.08697252 6721.43861955   97.3807169   231.15727538
  107.15710686  264.8843305   331.15662298] 7821.525595326531
SRA is Optimize    
best_position, best_fitness    [ 100.1325844  1902.41246195 6137.59931703   33.41046224  254.5614411
  293.46374878  178.82796345  354.54301714] 8140.144363374393
SRA is Optimize    
best_position, best_fitness    [ 520.82722072 1570.6586312  

# MPA

In [3]:
import numpy as np
import math
import random
def initialization(SearchAgents_no, dim, ub, lb):
    # Convert bounds to numpy arrays if they're lists
    ub = np.array(ub)
    lb = np.array(lb)
    # Initialize the positions of search agents
    Positions = np.random.uniform(0, 1, (SearchAgents_no, dim)) * (ub - lb) + lb
    return Positions

def MPA(objf,lb,ub,dim,SearchAgents_no,Max_iter,function_name):

    # Initialize the positions of search agents
    Prey = initialization(SearchAgents_no, dim, ub, lb)
    
    Top_predator_pos = np.zeros(dim)
    Top_predator_fit = np.inf

    Convergence_curve = np.zeros(Max_iter)
    stepsize = np.zeros((SearchAgents_no, dim))
    fitness = np.full((SearchAgents_no, 1), np.inf)

    
    Iter = 0
    FADs = 0.2
    P = 0.5

    print('MPA is optimizing  "' + function_name + '"')

    
    while Iter < Max_iter:
        # Detecting top predator
        for i in range(Prey.shape[0]):
            # Check if moths go out of the search spaceand bring it back
            for j in range(dim):
                Prey[i, j] = np.clip(Prey[i, j], lb[j], ub[j])
            fitness[i, 0] = objf(Prey[i, :])

            if fitness[i, 0] < Top_predator_fit:
                Top_predator_fit = fitness[i, 0]
                Top_predator_pos = Prey[i, :]

        # Marine Memory saving
        if Iter == 0:
            fit_old = fitness
            Prey_old = Prey

        Inx = fit_old < fitness
        Indx = np.tile(Inx, (1, dim))
        Prey = Indx * Prey_old + ~Indx * Prey
        fitness = Inx * fit_old + ~Inx * fitness

        fit_old = fitness
        Prey_old = Prey

        Elite = np.tile(Top_predator_pos, (SearchAgents_no, 1))
        CF = (1 - Iter / Max_iter) ** (2 * Iter / Max_iter)

        RB = np.random.randn(SearchAgents_no, dim)
        for i in range(Prey.shape[0]):
            RL = 0.05 * Levy(dim)
            for j in range(Prey.shape[1]):
                R = random.uniform(0,1)
                # Phase 1
                if Iter < Max_iter / 3:
                    stepsize[i, j] = RB[i, j] * (Elite[i, j] - RB[i, j] * Prey[i, j])
                    Prey[i, j] = Prey[i, j] + P * R * stepsize[i, j]
                # Phase 2
                elif Iter > Max_iter / 3 and Iter < 2 * Max_iter / 3:
                    if i > Prey.shape[0] / 2:
                        stepsize[i, j] = RB[i, j] * (RB[i, j] * Elite[i, j] - Prey[i, j])
                        Prey[i, j] = Elite[i, j] + P * CF * stepsize[i, j]
                    else:
                        stepsize[i, j] = RL[j] * (Elite[i, j] - RL[j] * Prey[i, j])
                        Prey[i, j] = Prey[i, j] + P * R * stepsize[i, j]
                # Phase 3
                else:
                    stepsize[i, j] = RL[j] * (RL[j] * Elite[i, j] - Prey[i, j])
                    Prey[i, j] = Elite[i, j] + P * CF * stepsize[i, j]

        
            if Iter == 0:
                fit_old = fitness
                Prey_old = Prey

            Inx = fit_old < fitness
            Indx = np.tile(Inx, (1, dim))
            Prey = Indx * Prey_old + ~Indx * Prey
            fitness = Inx * fit_old + ~Inx * fitness

            fit_old = fitness
            Prey_old = Prey
        # Eddy formation and FADs effect
            U1 = np.random.rand(SearchAgents_no, dim)
            U =np.where(U1 > 0.2, 1, 0)
            # Check if moths go out of the search spaceand bring it back
            for j in range(dim):
                r = np.random.rand()
                if r<0.2:
                   Prey[i,j] = Prey[i,j] + CF * ((lb[j] + np.random.rand() * (ub[j] - lb[j])) * U[i,j])
          
                else:
                   r1 = np.random.rand()
                   # select 3 random solution except current solution
                   ids_except_current = [_ for _ in range(SearchAgents_no) if _ != i]
                   id_1, id_2= random.sample(ids_except_current, 2)

                   stepsize[i,j] = (FADs * (1 - r1) + r1) * (Prey[id_1, j] - Prey[id_2, j])
                   Prey[i,j]= Prey[i,j] + stepsize[i,j]
                

        Iter = Iter + 1
        Convergence_curve[Iter - 1] = Top_predator_fit
#         if Iter % 1 == 0:
#             print(["At iteration " + str(Iter) + " the best fitness is " + str(Top_predator_fit)])

    return Top_predator_pos,Top_predator_fit,Convergence_curve



def Levy(dim):
    beta=1.5
    sigma=(math.gamma(1+beta)*math.sin(math.pi*beta/2)/(math.gamma((1+beta)/2)*beta*2**((beta-1)/2)))**(1/beta) 
    u= 0.01*np.random.randn(dim)*sigma
    v = np.random.randn(dim)
    zz = np.power(np.absolute(v),(1/beta))
    step = np.divide(u,zz)
    return step

In [4]:
# Define the G10 Fitness Function with Constraints
def g10_fitness(x):
    """
    Calculate the fitness value and constraints of G10.
    
    Parameters:
        x (array): Solution vector x with 8 decision variables.
        
    Returns:
        f (float): Fitness value of the solution.
        g (list): Constraint violation values (inequality constraints).
    """
    # Objective function
    f = x[0] + x[1] + x[2]
    
    # Constraints
    g = [
        -1 + 0.0025 * (x[3] + x[5]),
        -1 + 0.0025 * (x[4] + x[6] - x[3]),
        -1 + 0.01 * (x[7] - x[4]),
        -x[0] * x[5] + 833.33252 * x[3] + 100 * x[0] - 83333.333,
        -x[1] * x[6] + 1250 * x[4] + x[1] * x[3] - 1250 * x[3],
        -x[2] * x[7] + 1250000 + x[2] * x[4] - 2500 * x[4]
    ]
    
    return f, g

# Objective Function with Penalty for Constraints
def evaluate(x):
    f, g = g10_fitness(x)
    penalty = 1e6 * sum(max(0, violation) for violation in g)  # Penalty for any constraint violation
    return f + penalty


# G10 Problem Parameters
dim = 8
lb = [100, 1000, 1000, 10, 10, 10, 10, 10]
ub = [10000, 10000, 10000, 1000, 1000, 1000, 1000, 1000]
PopSize = 50
iters = 2000
runs = 30

# Run MPA on G10 and count constraint violations
constraint_violations = 0

for run in range(runs):
    # Initialize and run MPA optimization
    best_position, best_fitness,_  = MPA(evaluate, lb, ub, dim, PopSize, iters, function_name="G10")
    # best_position, best_fitness,_ = sra.optimize()
    print("best_position, best_fitness   ",  best_position, best_fitness)
    # Check constraint violations in the best solution found
    _, g = g10_fitness(best_position)
    
    # Count if any constraint is violated
    if any(violation > 0 for violation in g):
        constraint_violations += 1

print(f"Out of {runs} runs, constraints were violated {constraint_violations} times.")

MPA is optimizing  "G10"
best_position, best_fitness    [5239.66045072 1740.12757187 3839.07023556  271.93061115  346.43984512
  128.03334178  325.48502523  446.43967102] 10818.858258157356
MPA is optimizing  "G10"
best_position, best_fitness    [ 273.9882947  2204.25196945 4768.12397877  149.48042643  309.33523304
  250.49941479  240.1342478   409.33311226] 7246.364242920226
MPA is optimizing  "G10"
best_position, best_fitness    [ 417.5754366  1004.03039194 5746.90418451  165.83034517  270.1250636
  231.38718415  295.69001828  370.12456989] 7168.510013049773
MPA is optimizing  "G10"
best_position, best_fitness    [ 100.0087566  1502.94462871 5699.3118443   118.17161379  272.03228006
  280.86878571  246.13910241  372.03164477] 7302.265229619839
MPA is optimizing  "G10"
best_position, best_fitness    [ 115.205816   2550.10591861 4853.92057307  113.88764274  305.84552684
  200.6197523   208.03471782  405.84525197] 7519.232307687253
MPA is optimizing  "G10"
best_position, best_fitness   

# DE

In [9]:
# DE from the reference that the reviwer mentioned
import numpy as np

# Define the G10 Fitness Function with Constraints
def g10_fitness(x):
    """
    Calculate the fitness value and constraints of G10.
    
    Parameters:
        x (array): Solution vector x with 8 decision variables.
        
    Returns:
        f (float): Fitness value of the solution.
        g (list): Constraint violation values (inequality constraints).
    """
    # Objective function
    f = x[0] + x[1] + x[2]
    
    # Constraints
    g = [
        -1 + 0.0025 * (x[3] + x[5]),
        -1 + 0.0025 * (x[4] + x[6] - x[3]),
        -1 + 0.01 * (x[7] - x[4]),
        -x[0] * x[5] + 833.33252 * x[3] + 100 * x[0] - 83333.333,
        -x[1] * x[6] + 1250 * x[4] + x[1] * x[3] - 1250 * x[3],
        -x[2] * x[7] + 1250000 + x[2] * x[4] - 2500 * x[4]
    ]
    
    return f, g

# Objective function with penalty for constraints
def evaluate(x):
    f, g = g10_fitness(x)
    penalty = 1e6 * sum(max(0, violation) for violation in g)  # Penalty for any constraint violation
    return f + penalty

# Differential Evolution parameters
nVar = 8
VarSize = (nVar,)
VarMin = [100, 1000, 1000, 10, 10, 10, 10, 10]
VarMax = [10000, 10000, 10000, 1000, 1000, 1000, 1000, 1000]

MaxIt = 1000
nPop = 50
beta_min = 0.2
beta_max = 0.8
pCR = 0.2

# Number of runs for the DE optimization
runs = 30
constraint_violations = 0

# Run DE optimization multiple times and check constraint violations
for run in range(runs):
    # Initialization of individuals
    class Individual:
        def __init__(self):
            self.Position = None
            self.Cost = None

    BestSol = Individual()
    BestSol.Cost = float('inf')

    pop = [Individual() for _ in range(nPop)]
    for i in range(nPop):
        pop[i].Position = np.random.uniform(VarMin, VarMax, VarSize)
        pop[i].Cost = evaluate(pop[i].Position)
        
        if pop[i].Cost < BestSol.Cost:
            BestSol = pop[i]

    BestCost = np.zeros(MaxIt)

    # DE Main Loop
    for it in range(MaxIt):
        for i in range(nPop):
            x = pop[i].Position
            
            # Select three random individuals (excluding the current individual)
            A = np.random.permutation(nPop)
            A = A[A != i]
            a, b, c = A[:3]
            
            # Mutation
            beta = np.random.uniform(beta_min, beta_max, VarSize)
            y = pop[a].Position + beta * (pop[b].Position - pop[c].Position)
            y = np.clip(y, VarMin, VarMax)
            
            # Crossover
            z = np.zeros(x.shape)
            j0 = np.random.randint(len(x))
            for j in range(len(x)):
                if j == j0 or np.random.random() <= pCR:
                    z[j] = y[j]
                else:
                    z[j] = x[j]
            
            # Evaluate new solution
            NewSol = Individual()
            NewSol.Position = z
            NewSol.Cost = evaluate(NewSol.Position)
            
            # Selection
            if NewSol.Cost < pop[i].Cost:
                pop[i] = NewSol
                
                if pop[i].Cost < BestSol.Cost:
                    BestSol = pop[i]
        
        # Update Best Cost
        BestCost[it] = BestSol.Cost
        

    # Check constraint violations for the best solution found in this run
    print("best_position, best_fitness   ",   NewSol.Position, BestSol.Cost)
    _, g = g10_fitness(BestSol.Position)
    if any(violation > 0 for violation in g):
        constraint_violations += 1

# Display results
print(f"Out of {runs} runs, constraints were violated {constraint_violations} times.")


best_position, best_fitness    [ 100.         3128.38197502 4986.04310463  151.44652099  302.83115659
  171.90147964  163.54689983  402.20590525] 7731.604069890864
best_position, best_fitness    [2456.78112888 1000.         6464.59697567  227.84363058  256.18856715
  186.47443036  254.30182985  363.17949882] 7714.031813434982
best_position, best_fitness    [ 670.90406984 1734.00351732 5560.80408498  212.67845308  369.28713354
  198.58415364  213.86861411  447.34534078] 7679.138551077992
best_position, best_fitness    [1730.44272072 3352.74185389 5124.34284398   10.          311.65286038
  162.50249998  301.48800976  405.15659377] 7684.610171268141
best_position, best_fitness    [2356.3872766  3815.54243486 3838.6706352   148.74543524  334.12618869
  157.5968572   188.63369593  448.59988397] 7869.9793284868065
best_position, best_fitness    [ 399.05946934 3355.55829332 5439.32951652  151.29540849  300.6551216
  226.55455684  195.04495259  400.62135069] 7675.5286999366
best_position, bes

# GWO

In [4]:
#GWO
import numpy as np

# Define the G10 Fitness Function with Constraints
def g10_fitness(x):
    """
    Calculate the fitness value and constraints of G10.
    
    Parameters:
        x (array): Solution vector x with 8 decision variables.
        
    Returns:
        f (float): Fitness value of the solution.
        g (list): Constraint violation values (inequality constraints).
    """
    # Objective function
    f = x[0] + x[1] + x[2]
    
    # Constraints
    g = [
        -1 + 0.0025 * (x[3] + x[5]),
        -1 + 0.0025 * (x[4] + x[6] - x[3]),
        -1 + 0.01 * (x[7] - x[4]),
        -x[0] * x[5] + 833.33252 * x[3] + 100 * x[0] - 83333.333,
        -x[1] * x[6] + 1250 * x[4] + x[1] * x[3] - 1250 * x[3],
        -x[2] * x[7] + 1250000 + x[2] * x[4] - 2500 * x[4]
    ]
    
    return f, g

# Objective Function with Penalty for Constraints
def evaluate(x):
    f, g = g10_fitness(x)
    penalty = 1e6 * sum(max(0, violation) for violation in g)  # Penalty for any constraint violation
    return f + penalty

# Grey Wolf Optimizer (GWO) Implementation
class GreyWolfOptimizer:
    def __init__(self, obj_func, lb, ub, dim, pop_size=50, max_iter=2000):
        self.obj_func = obj_func
        self.lb = np.array(lb)
        self.ub = np.array(ub)
        self.dim = dim
        self.pop_size = pop_size
        self.max_iter = max_iter

        # Initialize population
        self.population = np.random.uniform(self.lb, self.ub, (pop_size, dim))
        self.fitness = np.array([self.obj_func(ind) for ind in self.population])

        # Initialize alpha, beta, and delta wolves
        self.alpha_pos = np.zeros(dim)
        self.beta_pos = np.zeros(dim)
        self.delta_pos = np.zeros(dim)
        self.alpha_score = float("inf")
        self.beta_score = float("inf")
        self.delta_score = float("inf")

    def optimize(self):
        for iteration in range(self.max_iter):
            # Update alpha, beta, and delta wolves
            for i in range(self.pop_size):
                score = self.fitness[i]
                if score < self.alpha_score:
                    self.delta_score, self.delta_pos = self.beta_score, self.beta_pos.copy()
                    self.beta_score, self.beta_pos = self.alpha_score, self.alpha_pos.copy()
                    self.alpha_score, self.alpha_pos = score, self.population[i].copy()
                elif score < self.beta_score:
                    self.delta_score, self.delta_pos = self.beta_score, self.beta_pos.copy()
                    self.beta_score, self.beta_pos = score, self.population[i].copy()
                elif score < self.delta_score:
                    self.delta_score, self.delta_pos = score, self.population[i].copy()

            # Update the positions of wolves
            a = 2 - iteration * (2 / self.max_iter)  # Decreasing factor
            for i in range(self.pop_size):
                for j in range(self.dim):
                    # Calculate D_alpha, D_beta, D_delta
                    r1, r2 = np.random.rand(), np.random.rand()
                    A1, C1 = 2 * a * r1 - a, 2 * r2
                    D_alpha = abs(C1 * self.alpha_pos[j] - self.population[i, j])
                    X1 = self.alpha_pos[j] - A1 * D_alpha

                    r1, r2 = np.random.rand(), np.random.rand()
                    A2, C2 = 2 * a * r1 - a, 2 * r2
                    D_beta = abs(C2 * self.beta_pos[j] - self.population[i, j])
                    X2 = self.beta_pos[j] - A2 * D_beta

                    r1, r2 = np.random.rand(), np.random.rand()
                    A3, C3 = 2 * a * r1 - a, 2 * r2
                    D_delta = abs(C3 * self.delta_pos[j] - self.population[i, j])
                    X3 = self.delta_pos[j] - A3 * D_delta

                    # Update position
                    self.population[i, j] = (X1 + X2 + X3) / 3

                # Apply boundary constraints
                self.population[i] = np.clip(self.population[i], self.lb, self.ub)

            # Update fitness
            self.fitness = np.array([self.obj_func(ind) for ind in self.population])

        return self.alpha_pos, self.alpha_score

# G10 Problem Parameters
dim = 8
lb = [100, 1000, 1000, 10, 10, 10, 10, 10]
ub = [10000, 10000, 10000, 1000, 1000, 1000, 1000, 1000]
PopSize = 50
iters = 2000
runs = 30

# Run GWO on G10 and count constraint violations
constraint_violations = 0

for run in range(runs):
    # Initialize and run GWO optimization
    gwo = GreyWolfOptimizer(obj_func=evaluate, lb=lb, ub=ub, dim=dim, pop_size=PopSize, max_iter=iters)
    best_position, best_fitness = gwo.optimize()
    print("best_position, best_fitness   ",  best_position, best_fitness)
    # Check constraint violations in the best solution found
    _, g = g10_fitness(best_position)
    
    # Count if any constraint is violated
    if any(violation > 0 for violation in g):
        constraint_violations += 1

print(f"Out of {runs} runs, constraints were violated {constraint_violations} times.")


best_position, best_fitness    [ 101.84093813 1682.71540722 5881.05934104   83.83174025  264.95540451
   17.90459891  218.59384803  364.90231917] 7665.615686391753
best_position, best_fitness    [ 445.24223721 1794.71297277 5094.61094515  149.55794269  296.55138644
  203.01260744  252.48663266  396.51420798] 7334.566155136028
best_position, best_fitness    [ 148.5852426  1965.74208837 6234.40691462   17.13541135  250.8684558
   39.56024541  166.02537955  350.83649491] 8348.734245595851
best_position, best_fitness    [ 137.7237155  2350.48142174 5818.47821414   18.82474071  267.51667905
   28.94011417  151.27289328  367.41439435] 8306.683351380694
best_position, best_fitness    [ 114.69898091 1523.36991284 5736.49913985  114.14476608  270.85290004
  230.50137626  242.85512085  370.81405271] 7374.568033605481
best_position, best_fitness    [ 434.42592639 1090.46858908 6128.05972646  129.97522834  254.95649904
  172.65335871  273.92204023  354.9448754 ] 7652.954241931529
best_position, be